SLiMe is a package that allows the user to simulate large-scale genomic data under realistically complex demographic histories while retaining information about local ancestry.

This notebook describes how to use SLiMe.

### Basic procedure

1. Recent history since the start of admixture is simulated forward-in-time with `SLiM`.
2. Older history of the ancestral populations is simulated backwards-in-time with `msprime`.

### Packages

In [1]:
import slime, msprime, pyslim

(Note: For now I've installed my local version of tskit that allows unary nodes to kept while simplifying - but use the proper version once the next tskit release is out!)

In [2]:
import sys
!{sys.executable} -m pip install tskit-0.1.6.dev0.tar.gz

Processing ./tskit-0.1.6.dev0.tar.gz
  Running setup.py bdist_wheel for tskit ... done
  Stored in directory: /Users/gtsambos/Library/Caches/pip/wheels/ee/67/b5/0a568d34bc8bb222f35ac2c0dd4e3152f678f1595763d27df1
Successfully built tskit
  Found existing installation: tskit 0.1.6.dev0
    Uninstalling tskit-0.1.6.dev0:
      Successfully uninstalled tskit-0.1.6.dev0


In [3]:
import tskit

## Inputs

**Necessary - SLiM script**

You must input a SLiM script that simulates the history of your admixed sample from the start of admixture until the present day.

See the [SLiM Manual](http://benhaller.com/slim/SLiM_Manual.pdf) for how to use the Eidos scripting language.

For the purposes of this notebook, here's a script for a simple two-way admixture under neutrality.

In [4]:
slim_script = '''
// set up a simple neutral simulation
initialize() {
	
	// Set constants.
	defineConstant("chromLength", 49999999);
	defineConstant("p0Size", 1000);
	defineConstant("p1Size", 1000);
	defineConstant("p2Size", 1000);
	defineConstant("p0Prop", 10/100);
	defineConstant("p1Prop", 1 - p0Prop);

	initializeSLiMModelType("WF");
	initializeSLiMOptions(keepPedigrees = T); // Keep pedigree information
	initializeTreeSeq();
	initializeMutationRate(0);
	
	// m1 mutation type: neutral
	initializeMutationType("m1", 0.5, "f", 0.0);
	
	// g1 genomic element type: uses m1 for all mutations
	initializeGenomicElementType("g1", m1, 1.0);
	
	// uniform chromosome of length 100 kb
	initializeGenomicElement(g1, 0, chromLength);
	
	// read in chr22 recombination map.
	// See example 6.1.2 in the SLiM manual.
	lines = readFile("examples/chr22.recombination_map");
	lines = lines[1:(size(lines)-1)]; // remove header
	rates = NULL;
	ends = NULL;
	
	for (line in lines)
	{
		components = strsplit(line, " ");
		ends = c(ends, asInteger(components[1]));
		rates = c(rates, asFloat(components[2]));
	}
	
	ends = ends - 1; // final coordinate in file is 50 000 000
	rates = rates * 1e-8;
	
	initializeRecombinationRate(rates, ends);
	
}

1 early(){
	sim.addSubpop("p0", p0Size);
	sim.addSubpop("p1", p1Size);

//	sim.treeSeqRememberIndividuals(sim.subpopulations.individuals);
}

1 late() { 
	
	// Add admixing population.
	sim.addSubpop("p2", p2Size);
	p2.setMigrationRates(c(p0,p1),c(p0Prop, p1Prop));
	
}


2 late() {

	p2.setMigrationRates(c(p0,p1),c(0.0,0.0));

}


// save the tree output after 200 generations.
200 late() {
 	sim.treeSeqOutput("examples/recent-history.trees");
	sim.simulationFinished();
}
'''

slim_file = open("examples/slim_script.slim", "w")
slim_file.writelines(slim_script)
slim_file.close()

**Necessary - ancient population configurations, demographic history and recombination rate**

At this stage, only a uniform recombination rate can be used, though this may change in the future.

In [5]:
rho = 1e-8 # recombination rate. 


In [6]:
######################
# POPULATION HISTORY #
######################
N0 = 1e4 # population size
divergenceGen = 1000

# Population IDs
population_configurations = [
    # CEU
    msprime.PopulationConfiguration(
        initial_size = N0,
        growth_rate = 0),
    # YRI
    msprime.PopulationConfiguration(                                
        initial_size = N0,                                
        growth_rate = 0),
    # ASW - needed as a 'dummy'
    msprime.PopulationConfiguration(
        sample_size = 0,                                                              
        growth_rate = 0)
]

demographic_events = [
	msprime.MassMigration(
		time=divergenceGen,
		source = 1,
		destination = 0,
		proportion = 1.0),
	msprime.MigrationRateChange(time = divergenceGen,
		rate = 0)
]

**Optional - neutral mutation rate**

By default this is 0.

In [7]:
mu = 1e-8 # rate of neutral mutations

**Optional - Recombination map**

A recombination map for the admixed history can be given. 
The file should look like the one below.
Ensure that the final row of the recombination map corresponds to the length of the chromosome that you wish to simulate.

```
Chromosome position COMBINED_rate(cM/Mb) Genetic_Map(cM)
22 16051347 9.6640973708 0
22 16052618 9.7078062447 0.0123386217370137
22 16053624 9.7138922111 0.0221107973013803
22 16053659 9.716343506 0.0224508693240903
22 16053758 9.707808785 0.0234119423938053
22 16054713 9.5801157234 0.0325609529096523
22 16054960 9.5805112114 0.0349273391788681
.
.
.
22 49991555 0.1286194771 73.1460580463483
22 49991660 0.1286937712 73.1460715591943
22 49992695 0.1297177603 73.1462058170762
22 49993745 0.1310658564 73.1463434362254
22 49994152 0.1310658564 73.146396780029
22 49997158 0.1310658564 73.1467907639933
22 49997614 0.1310667948 73.1468505304517
22 49997706 0.1313729993 73.1468626167677
22 49997967 0.1313729993 73.1468969051205
22 50000000 0.1315909877 73.1470166529193
```

## How to use the package

In [8]:
import slime

First, initialise an `AdmixtureSimulation` object containing all the desired inputs.

In [9]:
mysim = slime.AdmixtureSimulation("examples/slim_script.slim", 
                                  populations_to_sample_from = [2], sample_sizes = [10],
                                  neutral_mutation_rate = mu,
                                  ancient_recombination_rate = rho,
                                ancient_population_configurations = population_configurations,
                                 ancient_demographic_events = demographic_events,
                                 out_file = "examples/out.trees")

Next, run the debugger. At the moment, this debugger must be run before the simulation. I may change this in the future, but I will strongly recommend users to run the debugger first as there are many tricksy ways that incorrectly-specified inputs could mess up the simulation, and the warnings outputted can be pretty inscrutable.

In [10]:
ts = mysim.debugger()


SLiM input file: examples/slim_script.slim
SLiM output file: examples/recent-history.trees
We are sampling:
- 10 individuals from population 2
Ancient demography:
Epoch: 0 -- 1000.0 generations
     start     end      growth_rate |     0        1        2    
   -------- --------       -------- | -------- -------- -------- 
0 |  1e+04    1e+04               0 |     0        0        0    
1 |  1e+04    1e+04               0 |     0        0        0    
2 |    1        1                 0 |     0        0        0    

Events @ generation 1000.0
   - Mass migration: lineages move from 1 to 0 with probability 1.0
   - Migration rate change to 0 everywhere
Epoch: 1000.0 -- inf generations
     start     end      growth_rate |     0        1        2    
   -------- --------       -------- | -------- -------- -------- 
0 |  1e+04    1e+04               0 |     0        0        0    
1 |  1e+04    1e+04               0 |     0        0        0    
2 |    1        1                 0 |  

After the debugger has run, the `go()` method can be used to generate an admixed tree sequence with the specified history.

In [11]:
ts1 = mysim.go()

Simulating recent history with SLiM...
Taking samples from present day populations...
Simulating ancient history with msprime...
Adding variation...


Note that if desired, this method can be used several times to obtain replications.

# Individual steps: what's going on under the hood?

Here are the individual processes that happen inside this method.

## Step 1: Simulate recent history forward-in-time with SLiM.

The `simulate_recent_history` function is a wrapper for running SLiM via python.

In [12]:
slime.simulate_recent_history("examples/slim_script.slim", outFile = "examples/recent-history.trees",
                             logFile = "examples/recent-history.log")

At the end of the SLiM script, you must save the output as a tree sequence using the `TreeSeqOutput` function. The debugger will return an error if you don't.

*Could add: a simple function that adds this line into the SLiM script if it isn't in there already.*

## Step 2 (optional): subsample population

`SLiM` generates a tree sequence that holds the chromosomes of *entire populations*. If you are interested in only a subset of the simulated population, this tree sequence will contain a lot of redundant history. 

You can use `slime` to choose a random sample of individuals from the generated tree sequence and simplify the tree sequence so that only the history relevant to these samples is retained.

*To add: allow users to input any arbitrary list of nodes to use as samples.*

In [13]:
ts = tskit.load("examples/recent-history.trees")

In [14]:
new_ts = slime.TreeSequenceToSample(
    ts = ts,
    populations_to_sample_from = [2],
    sample_sizes = [10]
)

In [15]:
ts = new_ts.subsample()

## Step 3: recapitation

Next, we add the ancient history of the samples. This is generated with `msprime`.
This step is basically just a wrapper for the `recapitate` function in `pyslim`.

To do this, we need to add in some parameters pertaining to the ancient history of our sample.

Note that, atm, you must specify a uniform recombination rate for the ancient history. This [may change in the future](https://github.com/tskit-dev/pyslim/blob/master/README.md#things-you-cannot-do).

In [17]:
ts1 = pyslim.SlimTreeSequence.load_tables(ts.tables)


In [18]:
recap = ts1.recapitate(
	recombination_rate=rho,
	population_configurations = population_configurations,
	demographic_events=demographic_events,
    keep_first_generation = True
	)

## Step 4: add in variation

For the moment, let's assume all variation is neutral.

*To add: allow non-neutral mutations that are generated in SLiM to be removed before recapitation with msprime, then added back in this step.*

In [19]:
mut_ts = pyslim.SlimTreeSequence(msprime.mutate(recap, rate=mu, keep=True))